In [1]:
import os
import re
import json
import pandas as pd
from src.utils import read_config
from prompts import gemini_prompts
import google.generativeai as genai
import google.ai.generativelanguage as glm

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS, DistanceStrategy
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

from neo4j import GraphDatabase

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Building prefix dict from c:\Users\sean.chang\yfy\creative_corner\project_envelope\backend\dict\dict.big.txt ...
Loading model from cache C:\Users\SEAN~1.CHA\AppData\Local\Temp\jieba.u2b61ce2d1f177c74d958b0d3d402c57d.cache
Loading model cost 1.321 seconds.
Prefix dict has been built successfully.
c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
organized_result = pd.read_csv('temp_result/organized_result_demo.csv')
organized_result.head()

,document,target,reason,fact,keywords
0,(公布版)中科院人員回聘案-糾正案文-(112國正6)_20230927160316477.pdf,國防部,由國防部為國家中山科學研究院（下稱中科院）之監督機關，未能積極監督該院落實進用高齡人員實施計...,本案緣於近年審計部查核報告指出中科院行政法人化後衍生人事進用公正性及採購作業違反利益迴避等缺...,"['中科院', '未能', '計畫', '建立', '督促', '利益', '進用', '主..."
1,(公布版)糾正案文(111國正3)_20231124100827470.pdf,國防部暨所屬空軍司令部,由國防部軍售案涉及預算排擠效應及兵力整建計畫建案作業程序，爰以特別預算編列，採緊急應變項次非...,本案經調查竣事，確有下列失當之處，茲將事實及2理由臚列如後一、國防部軍售案涉及預算排擠效應及...,"['需求', '管理', '軍購', '計畫', '品項', '建案', '年度', '工項..."
2,(公布版)陸軍官校案-糾正案文(112國正5)_20230818155815896.pdf,陸軍軍官學校,由陸軍軍官學校於109年10月7日晚間發生學弟以為學長慶生為由，由2名學弟壓著學長進入浴室洗...,有關「網傳陸軍官校學生疑似霸凌影片，畫面中2名學弟壓著學長洗澡，過程中拍打屁股、熱水澆淋身體...,"['影片', '案件', '事件', '屁股', '予以', '作息', '全裸', '制止..."
3,(公布版）空軍女中校糾正案（111國正2）_20230417101815845.pdf,國防部空軍司令部,由國防部空軍司令部未徵得所屬幹部自由意願，違法指派其擔任與軍人身分無關之差勤，擔任退休將領眷...,國防部空軍司令部指派一位女性中校軍官到醫院擔任退休將領配偶的醫療看護，並核予公假等情乙案，案...,"['退休', '公假', '工作', '指派', '差假', '相符', '查與', '顯有..."
4,(定稿)112司正0002_維民案糾正案文-公布版112.08.21 (修正3處112091...,行政院,由威權統治時期國家機關對於海外歸國學人返國服務進行安全查核，依調查局所查復維民專案不予介聘之...,有關「據訴，法務部調查局曾以維民專案對渠進行偵防調查，復製作不實調查報告，影響渠聲譽及求職，...,"['介聘', '人民', '不予', '政治', '思想', '箝制', '權之', '工作..."


In [4]:
configs = read_config('.env/configs.json')
api_key =configs['g_key']
os.environ["GOOGLE_API_KEY"] = api_key
genai.configure(api_key=api_key)

In [5]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [6]:
kg_reseult = organized_result.copy()
for i, row in kg_reseult.iterrows():
    
    triple_prompt = PromptTemplate.from_template(gemini_prompts.TRIPLE_PROMPT)
    chain = triple_prompt | llm
    triple_data = {
        "reference": row['reason']
    }

    triple_response = chain.invoke(triple_data)

    triple_response_result = re.sub(r'[```json\n]', '', triple_response.content).replace("實體", "entity").replace("關係", "relationship").split("、")[0]

    to_json_prompt = PromptTemplate.from_template(gemini_prompts.TO_JSON_PROMPT)
    chain = to_json_prompt | llm
    data = {
        "reference": triple_response_result
    }

    to_json_response = chain.invoke(data)
    
    
    triple_json_result = re.sub(r'[\n]','',to_json_response.content)

    # triple_json_result = json.loads(f'[{triple_json_result}]')
    
    kg_reseult.loc[kg_reseult.index==i, 'kg'] = str(triple_json_result)
    
    

c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Co

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 500,
    length_function= len
)

In [14]:
kg_reseult = kg_reseult.rename(columns={'document':'source'})

In [15]:
reason_loader = DataFrameLoader(kg_reseult, page_content_column='reason')
reason_data = reason_loader.load()

# target_loader = DataFrameLoader(organized_result, page_content_column='target')
# target_data = target_loader.load()

reason_documents = text_splitter.transform_documents(reason_data)
print("已為所有文件進行 chunk ",len(reason_documents),"筆")

已為所有文件進行 chunk  242 筆


In [16]:
reason_documents[0].metadata

{'source': '(公布版)中科院人員回聘案-糾正案文-(112國正6)_20230927160316477.pdf',
 'target': '國防部',
 'fact': '本案緣於近年審計部查核報告指出中科院行政法人化後衍生人事進用公正性及採購作業違反利益迴避等缺失，又媒體報導中科院大量回聘退休高階主管，淪肥貓院疑慮等情，已調查完畢國防部為中科院之監督機關，未能善盡監督責任，應予糾正促其注意改善，糾正之事實與理由如下一、中科院為因應營運急速擴張與各項重大任務推展，須以熟練人力即時投入工作，同時藉經驗傳承深化人才培育等需要，訂定進用高齡人員實施計畫，以曾在該院服務期間已核定為從事及參與機敏專案之涉密工項，或具有關鍵、高工藝、外部勞動市場稀少、短期內無法取得之專業技術者等退休人員列為聘任對象，2其立意良善；惟中科院卻一再打破該實施計畫所特別訂定高齡人員以不任主管職的原則，排擠現職人員升遷，打擊工作士氣，相關高齡回任人員雖於媒體揭露後已分別卸任主管職務，但已嚴重斲傷中科院形象，且動輒增設非常態之高階職缺予以安置，亦難杜因人設事之譏評又，竟有個別高齡回任者之議薪結果，違反該實施計畫規定，逾越原薪資之三分之二之上限，核其所為，已有不當濫用人事制度彈性、獨厚特定人之嫌，斷難以契約自由或並未違反法律等語卸責國防部為中科院之監督機關，未能積極監督該院落實進用高齡人員實施計畫辦理人事相關事務，以維該院正面形象與激勵現職人員之工作士氣，核有怠失(一)中科院依據勞動基準法第54條「勞工非有下列情形之一，雇主不得強制其退休一、年滿65歲者……」、中高齡者及高齡者就業促進法第28條「65歲以上勞工，雇主得以定期勞動契約僱用之」該院「定期契約人員管理作業規定」及「聘請顧問作業規章」等相關規定，由該院院長於109年2月17日主持之「定期契約及延後退休人力調節研處作為研討會議」後，於同年3月11日訂定「國家中山科學研究院進用高齡人員實施計畫」（下稱實施計畫），依該實施計畫之具體作法內容，對於中科院進用原即服務於該院之相關高齡人員係採「先退再聘」方式，屆滿65歲者，一律先辦理退休，結算退休金後，再視工作特性，合議契約類型及工作內容；現有延退人員至合約結束止，依前述規定辦理(二)根據上述實施計畫之僱用原則，包括１、區分為定期契約及顧問2種類型，定期契約以能提供專業技術實務工作者為主，惟行政

In [10]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
# vector_db_path = "./vector_db_2/"

# if not os.path.exists(vector_db_path):
#     os.makedirs(vector_db_path)
#     print(f"Created folder: {vector_db_path}")
    
# vector_store = FAISS.from_documents(reason_documents, embeddings, normalize_L2=True, distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT)
# vector_store.save_local(folder_path=vector_db_path)

In [17]:
kg_reseult

,source,target,reason,fact,keywords,kg
0,(公布版)中科院人員回聘案-糾正案文-(112國正6)_20230927160316477.pdf,國防部,由國防部為國家中山科學研究院（下稱中科院）之監督機關，未能積極監督該院落實進用高齡人員實施計...,本案緣於近年審計部查核報告指出中科院行政法人化後衍生人事進用公正性及採購作業違反利益迴避等缺...,"['中科院', '未能', '計畫', '建立', '督促', '利益', '進用', '主...","{ ""entity1"": ""國防部"", ""relationship"": ""未能積極監督""..."
1,(公布版)糾正案文(111國正3)_20231124100827470.pdf,國防部暨所屬空軍司令部,由國防部軍售案涉及預算排擠效應及兵力整建計畫建案作業程序，爰以特別預算編列，採緊急應變項次非...,本案經調查竣事，確有下列失當之處，茲將事實及2理由臚列如後一、國防部軍售案涉及預算排擠效應及...,"['需求', '管理', '軍購', '計畫', '品項', '建案', '年度', '工項...","{ ""entity1"": ""國防部"", ""relati"": ""軍售案涉及預算排擠效應"",..."
2,(公布版)陸軍官校案-糾正案文(112國正5)_20230818155815896.pdf,陸軍軍官學校,由陸軍軍官學校於109年10月7日晚間發生學弟以為學長慶生為由，由2名學弟壓著學長進入浴室洗...,有關「網傳陸軍官校學生疑似霸凌影片，畫面中2名學弟壓著學長洗澡，過程中拍打屁股、熱水澆淋身體...,"['影片', '案件', '事件', '屁股', '予以', '作息', '全裸', '制止...",
3,(公布版）空軍女中校糾正案（111國正2）_20230417101815845.pdf,國防部空軍司令部,由國防部空軍司令部未徵得所屬幹部自由意願，違法指派其擔任與軍人身分無關之差勤，擔任退休將領眷...,國防部空軍司令部指派一位女性中校軍官到醫院擔任退休將領配偶的醫療看護，並核予公假等情乙案，案...,"['退休', '公假', '工作', '指派', '差假', '相符', '查與', '顯有...","{ ""entity1"": ""國防部空軍司令部"", ""relationship"": ""指派..."
4,(定稿)112司正0002_維民案糾正案文-公布版112.08.21 (修正3處112091...,行政院,由威權統治時期國家機關對於海外歸國學人返國服務進行安全查核，依調查局所查復維民專案不予介聘之...,有關「據訴，法務部調查局曾以維民專案對渠進行偵防調查，復製作不實調查報告，影響渠聲譽及求職，...,"['介聘', '人民', '不予', '政治', '思想', '箝制', '權之', '工作...","{ ""entity1"": ""國家機關"", ""relationship"": ""進行安全查核..."
...,...,...,...,...,...,...
237,賴芳徵妨害公務案糾正案文20191216_20230117095815206.pdf,臺北市政府警察局,由賴芳徵於103年4月29日在立法院中興大樓出入口與數十群眾集會靜坐訴求反核四，被警方逮捕詎...,據賴芳徵陳訴，其於103年4月29日於立法院中興大樓出入口警方架設之拒馬前靜坐陳情，並企圖以...,"['賴芳徵', '中正', '警察局', '違失', '市政府', '分局', '提出', ...","{ ""entity1"": ""賴芳徵"", ""relationship"": ""集會靜坐訴求反..."
238,金門案糾正案文-上網版(111.3.9調整)(上網公布)_20231214150816333...,教育部、金門縣政府,由本案金門縣某家長分別自97、99、102及105學年度起至109學年度止，分別限制4名子女...,本院為調查「金門縣金東地區一戶家長，拒讓4名子女上學，長達11年，政府相關處理過程及措施，是...,"['依法', '接受', '核有', '利益', '權法', 'CRC', '政府', '情...","{ ""三元組 1"": { ""entity1"": ""金門縣某家長"", ""rela..."
239,電力系統三相不平衡糾正案(決議版)_20230615082314996.pdf,台灣電力公司,由台電公司過去配電作業未確實登載相別，配電圖資管理系統(DMMS)相別資料紊亂，相別量測設備...,本案緣於審計部民國（下同）109年度「中央政府總決算審核報告」提出「改善配電系統三相不帄衡，...,"['三相', '低估', '掌握', '改善', '量測', '情形', '圖資', '安培...","{ ""entity1"": ""台電公司"", ""relationship"": ""管理不當"",..."
240,高雄二監拒絕收監案糾正公告_20231110101816288.pdf,法務部矯正署、法務部矯正署高雄第二監獄、臺灣橋頭地方檢察署,由法務部矯正署對於各監獄所「啟動」拒絕收監申請案之裁量標準及空間欠缺一致性之標準，且「審核」...,本案緣據報載，法務部矯正署高雄第二監獄（下稱高雄二監）新收受刑人李○邦(下稱李員)，於新收入...,"['李員', '牟利', '法律', '商品', '政府', '缺乏', '直播', '得以...","{ ""entity1"": ""法務部矯正署"", ""relationship"": ""制定"",..."


In [18]:
id_list = kg_reseult['source'].to_list()

In [19]:
vector_db_path = "./chroma_db/"
chroma_collection_name = "corretion_v2"
if not os.path.exists(vector_db_path):
    os.makedirs(vector_db_path)
    print(f"Created folder: {vector_db_path}")
    
vector_store = Chroma.from_documents(documents=reason_documents, 
                                     persist_directory=vector_db_path, 
                                     collection_name=chroma_collection_name, 
                                     ids=id_list,
                                     embedding=embeddings)

In [20]:
vectorstore = Chroma(persist_directory=vector_db_path, collection_name=chroma_collection_name, embedding_function=embeddings)
vectorstore._collection.count()

242

In [105]:
# vector_store.delete_collection()

In [7]:
kg_reseult.to_csv('temp_result/result_with_kg.csv', index=False)

In [83]:
def transform_kg_to_json(df, column_name):
    def validate_and_transform(kg_value):
        try:
            triple_json_result = re.sub(r'[\n]', '', kg_value)
            triple_json_result = json.loads(f'[{triple_json_result}]')
            return triple_json_result
        except:
            return None

    df[f'{column_name}_json'] = df[column_name].apply(validate_and_transform)
    return df

In [168]:
df_test = kg_reseult[kg_reseult['kg_json'].notnull()]
df_test = transform_kg_to_json(df_test, 'kg')

C:\Users\sean.chang\AppData\Local\Temp\ipykernel_4404\2243806963.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{column_name}_json'] = df[column_name].apply(validate_and_transform)


In [169]:
URI = "bolt://localhost:7688"
AUTH = ("neo4j", "!QAZ2wsx")

with GraphDatabase.driver(URI, auth=AUTH, encrypted=False) as driver:
    driver.verify_connectivity()

In [214]:
def create_relationships(tx, data):
    # print(data)
    for item in data:
        try:
            tx.run("MERGE (a:Entity {name: $entity1}) "
                "MERGE (b:Entity {name: $entity2}) "
                "MERGE (a)-[r:RELATIONSHIP {type: $relationship}]->(b)",
                entity1=item[0]["entity1"], relationship=item[0]["relationship"], entity2=item[0]["entity2"])
        except:
            pass

In [212]:
row = df_test[df_test.index == 2]
data = row.kg_json
data

2    [{'entity1': '台北市政府', 'relationship': '未善盡', '...
Name: kg_json, dtype: object

In [213]:
for item in data:
    try:
        print(item)
        print(item[0]["entity1"])
        
    except:
        pass

[{'entity1': '台北市政府', 'relationship': '未善盡', 'entity2': '土地使用管制'}]
台北市政府


In [215]:
for i in df_test.index:
    row = df_test[df_test.index == i]
    data = row.kg_json
    with GraphDatabase.driver(uri=URI, auth=("neo4j", "neo4j")) as driver:
        with driver.session() as session:
            session.write_transaction(create_relationships, data)

In [216]:
def fetch_data(tx):
    result = tx.run("MATCH (n:Entity)-[r:RELATIONSHIP]->(m:Entity) "
                    "RETURN n.name, r.type, m.name")
    for record in result:
        print(f"{record['n.name']} {record['r.type']} {record['m.name']}")

with GraphDatabase.driver(uri=URI, auth=("neo4j", "neo4j")) as driver:  # Replace auth credentials as necessary
    with driver.session() as session:
        session.read_transaction(fetch_data)


南投縣草屯鎮公所 執行 過渡時期緊急垃圾處理計畫委託焚化處理案
嘉義市政府 辦理 嘉義市華興橋改建工程周邊道路(許厝庄12米計畫道路工程)
交通部臺灣鐵路管理局（臺鐵局） 辦理 「工程建設暨政策網路行銷採購」
衛生福利部澎湖醫院 未依政府採購法及契約規定 辦理107年停車場設備租賃等採購案
文化部所屬文化資產局 辦理 水下文化資產調查研究工作相關儀器設備案
雲林縣褒忠鄉公所 辦理 111年度「褒忠鄉大手牽小手為愛一起走」活動
空軍司令部 辦理 採購案
新竹縣新豐鄉公所 未審慎辦理 清潔隊人員出缺之甄補作業
台電公司 審議 立式車床採購規格
文化部 洽請 臺北市政府
花蓮區漁會 辦理 三機一體彩色漁探衛星導航電子海圖儀及船用DSB無線電對講機採購案
台北市政府 未善盡 土地使用管制
行政院國軍退除役官兵輔導委員會台北榮民總醫院 辦理 「醫學科技大樓法定停車場工程」興建發包作業
行政院國軍退除役官兵輔導委員會 未經蒐集商情資訊 十一所榮民醫院及十八所安養機構之污水處理廠及衛生下水道工程
教育部 辦理 二○○一年國際生物奧林匹亞競賽參賽計畫
卓蘭鎮公所 辦理 卓蘭鎮老庄區域排水震災修復工程
交通部台灣鐵路管理局 未能善盡職責詳實審查 廠商投標文件
台南市政府 怠未依計畫補助原則和程序 環保庫錢爐
國防部中山科學研究院 編列違反規定 國防部
中華郵政股份有限公司 違反 勞動基準法相關規定
科技部 排除 ○○公司
國立臺灣師範大學 辦理 林口校區資訊與教學大樓新建工程案
原能會 委託 清華大學
東華大學 未詳予審查 繼受廠商之履約資格
國防部 所屬 海軍司令部
國防部陸軍司令部 辦理 「步兵多武器射擊模擬系統」採購案
國防部 辦理 化學消防車採購案
新竹縣政府 接受補助 內政部營建署
南投縣政府 辦理 河川土石疏濬作業
台灣電力股份有限公司高雄興達電廠 辦理 「興#3
國軍退除役官兵輔導委員會花蓮縣榮民服務處 辦理 「100-101年度花蓮地區單身亡故榮民殯葬事務採購案」
台灣中油股份有限公司 採購 液化天然氣LNG
新竹縣五峰鄉公所 辦理 勞務及工程採購案
高雄縣鳳山市文山國民小學 違反 九年一貫課程綱要
財團法人國家實驗研究院國家太空中心主任吳作樂等 辦理 台灣自主衛星ARGO計畫
前行政院青年輔導委員會 辦理 「一百年青年政策大聯盟公共參與網站暨網路經營計畫」
行政院農

In [198]:
def clear_neo4j_database(uri, auth):
    driver = GraphDatabase.driver(uri, auth=auth)
    
    def clear_db(tx):
        tx.run("MATCH (n) DETACH DELETE n")

    with driver.session() as session:
        session.write_transaction(clear_db)
    
    driver.close()
clear_neo4j_database(URI, AUTH)